In [1]:
import os
import sqlite3
import csv

# Define the directory and database name
data_directory = "data"
db_name = "birds_db"
csv_file = "birddata.csv"

# Construct path for the database:
db_path = os.path.join(data_directory, db_name)

# Construct path for the csv file:
csv_path = os.path.join(data_directory, csv_file)

# Names of our desired columns from the CSV file:
selected_column_names = [ 
    "COMMON NAME", 
    "SCIENTIFIC NAME",
    "OBSERVATION COUNT",
    "COUNTRY",
    "STATE",
    "COUNTY",
    "LOCALITY",
    "LATITUDE",
    "LONGITUDE",
    "OBSERVATION DATE",
    "TIME OBSERVATIONS STARTED",
    "PROTOCOL TYPE",
    "PROTOCOL CODE",
    "REVIEWED" 
] 

# Read the selected columns from the CSV file
with open(csv_path, 'r') as file:
    reader = csv.reader(file)
    header = next(reader)  # Read the header row
    selected_column_indices = [header.index(col_name) for col_name in selected_column_names]

    rows = []
    for row in reader:
        subset_row = [row[i] for i in selected_column_indices]  # Extract the desired columns
        rows.append(subset_row)

# Connect to the database 
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Drop the table if it exists
cursor.execute("DROP TABLE IF EXISTS birds")

# Create the table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS birds (
        COMMON_NAME TEXT,
        SCIENTIFIC_NAME TEXT,
        OBSERVATION_COUNT TEXT,
        COUNTRY TEXT,
        STATE TEXT,
        COUNTY TEXT,
        LOCALITY TEXT,
        LATITUDE REAL,
        LONGITUDE REAL,
        OBSERVATION_DATE TEXT,
        TIME_OBSERVATIONS_STARTED TEXT,
        PROTOCOL_TYPE TEXT,
        PROTOCOL_CODE TEXT,
        REVIEWED TEXT
    )
''')

# Construct the SQL command using the selected column names
# 
# Replace spaces with underscores in the column names
desired_column_names = [col_name.replace(' ', '_') for col_name in selected_column_names]

# Join the desired columns
column_list = ', '.join(desired_column_names)
placeholders = ', '.join(['?' for _ in selected_column_names])
sql_command = f"INSERT INTO birds ({column_list}) VALUES ({placeholders})"
cursor.executemany(sql_command, rows)
conn.commit()
conn.close()
